### **Tiny Thesis** - Does ACT lean heavily on the “Leader vs Follower” Joint-Discrepancy for Force Modelling 
**Date:** 2026-02-20

**System:** Conda : act_train

**Goal:** See title 


### 1. Motivation

**Goal** : Show that ACT does rely very heavily on Joint-State Discrepancy to predict approriate next action, thus accomplishing Force-Aware actions like Plug insertion.

## 2. Task design and Hypothesis

### Task Description

The Suite of Tasks are:
1. Lego-Insertion
2. Plug-Insert_noAdjust

### Task Configurations
We perform the below tasks:
* TRAIN 1:
    * Model: ACT Base
    * Cam Views: 3 views
    * Dataset: Plug insert 30 eps

* TRAIN 2:
    * Model: ACT Base - Obs joints as Target
    * Cam Views: 3 views
    * Dataset: Plug insert 30 eps


## 3. Task Design Considerations
N.A

## 4. Method

NOTE: jupyter notebooks cannot prevent process kill even with nohup. We run below in a tmux terminal.

* TRAIN 1:
    * Model: ACT Base
    * Cam Views: 3 views
    * Dataset: Plug insert 30 eps

In [ ]:
# Train Base ACT

nohup python3 imitate_episodes.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_base \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 8 \
--dim_feedforward 3200 \
--num_epochs 6000 \
--lr 1e-5 --seed 0 \
> plug_insert_base.log 2>&1

* TRAIN 2:
    * Model: ACT Base - Obs joints as Target
    * Cam Views: 3 views
    * Dataset: Plug insert 30 eps

In [ ]:
python3 imitate_episodes.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_obs_only \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 8 \
--dim_feedforward 3200 \
--num_epochs 6000 \
--lr 1e-5 --seed 0 \
> plug_insert_base.log 2>&1

## 5. Results + Evaluation
We can only run real world rollouts for each of the tests.\
For each training configuration, we run 10+ rollouts to evaluate performance.

In [ ]:
python3 imitate_episodes_record_torque.py \
--task_name plug_insert \
--ckpt_dir ./ckpt_dir/plug_insert_base \
--robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
--policy_class ACT \
--kl_weight 10 \
--chunk_size 100 \
--hidden_dim 512 \
--batch_size 8 \
--dim_feedforward 3200 \
--num_epochs 6000 \
--lr 1e-5 --seed 0 \
--eval \
--temporal_agg

### Policy 2

In [ ]:
python3 imitate_episodes_record_torque.py \
  --task_name plug_insert \
  --ckpt_dir ./ckpt_dir/plug_insert_base_no_left \
  --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
  --policy_class ACT \
  --kl_weight 10 \
  --chunk_size 100 \
  --hidden_dim 512 \
  --batch_size 16 \
  --dim_feedforward 3200 \
  --num_epochs 6000 \
  --lr 1e-5 --seed 0 \
  --exclude_cameras camera_left_shoulder \
  --eval \
  --temporal_agg

### Policy 3

In [ ]:
python3 imitate_episodes_record_torque.py \
  --task_name plug_insert \
  --ckpt_dir ./ckpt_dir/plug_insert_base60 \
  --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
  --policy_class ACT \
  --kl_weight 10 \
  --chunk_size 100 \
  --hidden_dim 512 \
  --batch_size 16 \
  --dim_feedforward 3200 \
  --num_epochs 8000 \
  --lr 1e-5 --seed 0 \
  --eval \
  --temporal_agg

### Policy 4

In [ ]:
python3 imitate_episodes_record_torque.py \
  --task_name plug_insert \
  --ckpt_dir ./ckpt_dir/plug_insert_base60_no_left \
  --robot /home/khw/interbotix_ws/src/aloha/config/robot/aloha_solo \
  --policy_class ACT \
  --kl_weight 10 \
  --chunk_size 100 \
  --hidden_dim 512 \
  --batch_size 16 \
  --dim_feedforward 3200 \
  --num_epochs 8000 \
  --lr 1e-5 --seed 0 \
  --exclude_cameras camera_left_shoulder  \
  --eval \
  --temporal_agg

### Summary of results
**Conclusion: Yes, Top View DOES make a difference.**

* Strictly speaking, none of the policies performed above 10% if we want to use Binary Success/Failure.
* By eye test, it's easy to see using the Top Camera showed improvement.
* We could watch the results videos for reference.

### Limitations

* Potentially the High Cam could be in a more informative position.
* Testing with Positional generalization is too unstructured.

**Todo:**
* Create an image overlay system capturing Object Pose setup before each Episode Capture.

## Future Design considerations 
* When policy gets stuck, it should be able to reset back to a wider view position, dynamically recompute the temporal aggregation / temporal chunk size to allow reset.

## Open Questions / Next Steps
* What about other cameras views?